# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
J. Li  ->  J. Li  |  ['J. Li']
T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 68 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.05430


extracting tarball to tmp_2504.05430...

 done.


Found 87 bibliographic references in tmp_2504.05430/output.bbl.
Retrieving document from  https://arxiv.org/e-print/2504.05645


...

T. Suhasaria  ->  T. Suhasaria  |  ['T. Suhasaria']


/tmp/ipykernel_3241/2822249172.py:52: LatexWarning: 2504.06005 did not run properly
list index out of range
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2504.06118


extracting tarball to tmp_2504.06118...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.05430-b31b1b.svg)](https://arxiv.org/abs/2504.05430) | **An Investigation of Disk Thickness in M51 from H-alpha, Pa-alpha, and Mid-Infrared Power Spectra**  |
|| B. G. Elmegreen, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-04-09*|
|*Comments*| *20 pages, 7 figures, accepted by Astrophysical Journal April 5, 2025*|
|**Abstract**|            Power spectra (PS) of high-resolution images of M51 (NGC 5194) taken with the Hubble Space Telescope and the James Webb Space Telescope have been examined for evidence of disk thickness in the form of a change in slope between large scales, which map two-dimensional correlated structures, and small scales, which map three-dimensional correlated structures. Such a slope change is observed here in H-alpha, and possibly Pa-alpha, using average PS of azimuthal intensity scans that avoid bright peaks. The physical scale of the slope change occurs at ~120 pc and ~170 pc for these two transitions, respectively. A radial dependence in the shape of the H-alpha PS also suggests that the length scale drops from ~180 pc at 5 kpc, to ~90 pc at 2 kpc, to ~25 pc in the central ~kpc. We interpret these lengths as comparable to the thicknesses of the star-forming disk traced by HII regions. The corresponding emission measure is ~100 times larger than what is expected from the diffuse ionized gas. PS of JWST Mid-IR Instrument (MIRI) images in 8 passbands have more gradual changes in slope, making it difficult to determine a specific value of the thickness for this emission.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.05645-b31b1b.svg)](https://arxiv.org/abs/2504.05645) | **A Study of Multiple Molecular Lines at the 3 mm Band toward Gas Infalling Sources**  |
|| Y. Yang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-09*|
|*Comments*| **|
|**Abstract**|            The study of multiple molecular spectral lines in gas infalling sources can provide the physical and chemical properties of these sources and help us estimate their evolutionary stages. We report line detections within the 3 mm band using the FTS wide-sideband mode of the IRAM 30 m telescope toward 20 gas-infalling sources. Using XCLASS, we identify the emission lines of up to 22 molecular species (including a few isotopologues) and one hydrogen radio recombination line in these sources. H$^{13}$CO$^+$, HCO$^+$, HCN, HNC, c-C$_3$H$_2$, and CCH lines are detected in 15 sources. We estimate the rotation temperatures and column densities of these molecular species using the LTE radiative transfer model, and compare the molecular abundances of these sources with those from nine high-mass star-forming regions reported in previous studies and with those from the chemical model. Our results suggest that G012.79-0.20, G012.87-0.22 clump A and B, and G012.96-0.23 clump A may be in the high-mass protostellar object stage, while sources with fewer detected species may be in the earlier evolutionary stage. Additionally, the CCH and c-C$_3$H$_2$ column densities in our sources reveal a linear correlation, with a ratio of N(CCH)/N(c-C$_3$H$_2$) = 89.2$\pm$5.6, which is higher than the ratios reported in the literature. When considering only sources with lower column densities, this ratio decreases to 29.0$\pm$6.1, consistent with those of diffuse clouds. Furthermore, a comparison between the N(CCH)/N(c-C$_3$H$_2$) ratio and the sources' physical parameters reveals a correlation, with sources exhibiting higher ratios tending to have higher kinetic temperatures and H$_2$ column densities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06118-b31b1b.svg)](https://arxiv.org/abs/2504.06118) | **Dynamical Dark Energy in light of the DESI DR2 Baryonic Acoustic Oscillations Measurements**  |
|| G. Gu, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-04-09*|
|*Comments*| *14+9 pages, 5+7 figures*|
|**Abstract**|            We investigate whether dark energy deviates from the cosmological constant ($\Lambda$CDM) by analyzing baryon acoustic oscillation (BAO) measurements from the Data Release 1 (DR1) and Data Release 2 (DR2) of DESI observations, in combination with Type Ia supernovae (SNe) and cosmic microwave background (CMB) distance information. We find that with the larger statistical power and wider redshift coverage of the DR2 dataset the preference for dynamical dark energy does not decrease and remains at approximately the same statistical significance as for DESI~DR1. Employing both a shape-function reconstruction and non-parametric methods with a correlation prior derived from Horndeski theory, we consistently find that the dark energy equation of state $w(z)$ evolves with redshift. While DESI DR1 and DR2 BAO data alone provide modest constraints, combining them with independent SNe samples (PantheonPlus, Union3, and the DES 5-year survey) and a CMB distance prior strengthens the evidence for dynamical dark energy. Bayesian model-selection tests show moderate support for dark energy dynamics when multiple degrees of freedom in $w(z)$ are allowed, pointing to increasing tension with $\Lambda$CDM at a level of roughly $3\sigma$ (or more in certain data combinations). Although the methodology adopted in this work is different from those used in companion DESI papers, we find consistent results, demonstrating the complementarity of dark energy studies performed by the DESI collaboration. Although possible systematic effects must be carefully considered, it currently seems implausible that $\Lambda$CDM will be rescued by future high-precision surveys, such as the complete DESI, Euclid, and next-generation CMB experiments. These results therefore highlight the possibility of new physics driving cosmic acceleration and motivate further investigation into dynamical dark energy models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.06005-b31b1b.svg)](https://arxiv.org/abs/2504.06005) | **A deep search for Complex Organic Molecules toward the protoplanetary disk of V883 Ori**  |
|| A. M. A. Fadul, et al. -- incl., <mark>T. Suhasaria</mark> |
|*Appeared on*| *2025-04-09*|
|*Comments*| *42 pages, 27 figures, accepted for publication in Astronomical Journal (AJ)*|
|**Abstract**|            Complex Organic Molecules (COMs) in the form of prebiotic molecules are potentially building blocks of life. Using Atacama Large Millimeter/submillimeter Array (ALMA) Band 7 observations in spectral scanning mode, we carried out a deep search for COMs within the disk of V883 Ori, covering frequency ranges of $\sim$ 348 - 366 GHz. V883 Ori is an FUor object currently undergoing an accretion burst, which increases its luminosity and consequently increases the temperature of the surrounding protoplanetary disk, facilitating the detection of COMs in the gas phase. We identified 26 molecules, including 14 COMs and 12 other molecules, with first detection in this source of the molecules: CH3OD, H2C17O, and H213CO. We searched for multiple nitrogen-bearing COMs, as CH3CN had been the only nitrogen-bearing COM that has been identified so far in this source. We also detected CH3CN, and tentatively detect CH3CH2CN, CH2CHCN, CH3OCN, CH3NCO, and NH2CHO. We compared the abundances relative to CH3OH with those in the handful of objects with previous detections of these species: the Class 0 protostars IRAS 16293-2422 A, IRAS 16293-2422 B and B1-c, the high-mass star-forming region Sagittarius B2 (North), the Solar System comet 67P/Churyumov-Gerasimenko, and the protoplanetary disk of Oph-IRS 48. We report $\sim$ 1 to 3 orders of magnitude higher abundances compared to Class 0 protostars and $\sim$ 1 to 3 orders of magnitude lower abundances compared to the protoplanetary disk, Sagittarius B2 (North), and 67P/C-G. These results indicate that the protoplanetary disk phase could contribute to build up of COMs.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.05430/./feast_m51circle_dr3_cut_ha_pa_miri_slope_deriv_3panels.jpg', 'tmp_2504.05430/./m51_ha_eight_circles_dr3_screenshot_with_200pc_littlecircle.jpg', 'tmp_2504.05430/./m51_ha_four_circles_dr3_screenshot.jpg', 'tmp_2504.05430/./feast_m51_ha_circle_dr3_3_and_4.jpg']
copying  tmp_2504.05430/./feast_m51circle_dr3_cut_ha_pa_miri_slope_deriv_3panels.jpg to _build/html/
copying  tmp_2504.05430/./m51_ha_eight_circles_dr3_screenshot_with_200pc_littlecircle.jpg to _build/html/
copying  tmp_2504.05430/./m51_ha_four_circles_dr3_screenshot.jpg to _build/html/
copying  tmp_2504.05430/./feast_m51_ha_circle_dr3_3_and_4.jpg to _build/html/
exported in  _build/html/2504.05430.md
    + _build/html/tmp_2504.05430/./feast_m51circle_dr3_cut_ha_pa_miri_slope_deriv_3panels.jpg
    + _build/html/tmp_2504.05430/./m51_ha_eight_circles_dr3_screenshot_with_200pc_littlecircle.jpg
    + _build/html/tmp_2504.05430/./m51_ha_four_circles_dr3_screenshot.jpg
    + _build/html/tmp_2504.05430/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# An Investigation of Disk Thickness in M51 from H$\alpha$, Pa$\alpha$, and Mid-Infrared Power Spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.05430-b31b1b.svg)](https://arxiv.org/abs/2504.05430)<mark>Appeared on: 2025-04-09</mark> -  _20 pages, 7 figures, accepted by Astrophysical Journal April 5, 2025_

</div>
<div id="authors">

B. G. Elmegreen, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Power spectra (PS) of high-resolution images of M51 (NGC 5194) taken with the Hubble Space Telescope and the James Webb Space Telescope have been examined for evidence of disk thickness in the form of a change in slope between large scales, which map two-dimensional correlated structures, and small scales, which map three-dimensional correlated structures. Such a slope change is observed here in H $\alpha$ , and possibly Pa $\alpha$ , using average PS of azimuthal intensity scans that avoid bright peaks.  The physical scale of the slope change occurs at $\sim120$ pc and $\sim170$ pc for these two transitions, respectively.  A radial dependence in the shape of the H $\alpha$ PS also suggests that the length scale drops from $\sim180$ pc at 5 kpc, to $\sim90$ pc at 2 kpc, to $\sim25$ pc in the central $\sim$ kpc. We interpret these lengths as comparable to the thicknesses of the star-forming disk traced by HII regions.  The corresponding emission measure is $\sim100$ times larger than what is expected from the diffuse ionized gas. PS of JWST Mid-IR Instrument (MIRI) images in 8 passbands have more gradual changes in slope, making it difficult to determine a specific value of the thickness for this emission.

</div>

<div id="div_fig1">

<img src="tmp_2504.05430/./feast_m51circle_dr3_cut_ha_pa_miri_slope_deriv_3panels.jpg" alt="Fig4" width="100%"/>

**Figure 4. -** PS (top) and their running slopes (bottom) for H$\alpha$(left), Pa$\alpha$(middle), and eight mid-infrared passbands (right). The curves are shifted upward for clarity with slope zero-levels in the bottom panels indicated by circles.  For H$\alpha$ and Pa$\alpha$, the top curves in each panel have the lowest intensity limits and the smallest numbers of scans in the average PS, and the bottom curves in each panel have all the scans, excluding only those with negative intensity peaks. The 8 PS on the right correspond to the 8 near-infrared bands, as labeled in the bottom panel; each is from an average of the PS with the lowest intensity peaks.  A slight break at a size of $1/k\sim120$ pc is visible in the top three H$\alpha$ PS (at the vertical black line). A weaker break at $\sim170$ pc is in some of the Pa$\alpha$ PS. The mid-infrared PS on the right have a more gradual change in slope; there is no obvious break but a vertical line shows 100 pc. The running slopes in the bottom panels show sudden changes at the positions of the PS breaks. The horizontal black lines are least-square fits to the PS slopes at wavenumbers below the suggested breaks and at wavenumbers above the breaks and up to the value of $k/k_0$ corresponding to 5 times the FWHM of the PSF. The mid-infrared passbands on the right have little or no span of PS from the fiducial scale of 100 pc to the PSF. The cyan curve in the top left panel is the PS of an H$\alpha$ intensity scans with a bright point-like source.  Radial ranges for the PS are indicated in the top panel.
 (*fig:derivatives*)

</div>
<div id="div_fig2">

<img src="tmp_2504.05430/./m51_ha_eight_circles_dr3_screenshot_with_200pc_littlecircle.jpg" alt="Fig1.1" width="50%"/><img src="tmp_2504.05430/./m51_ha_four_circles_dr3_screenshot.jpg" alt="Fig1.2" width="50%"/>

**Figure 1. -** (Left:) Image of M51 in H$\alpha$ with concentric circles at sample radii of azimuthal intensity scans used to derive the PS. The circles are spaced by 600 pixels, which is 200 scans, $24^{\prime\prime}$, or 872 pc. A small circle midway between the 4th and 5th annuli below the center has a diameter corresponding to 200 pc. (Right:) H$\alpha$ with circles at $16^{\prime\prime}$, $35^{\prime\prime}$, $80^{\prime\prime}$, and $180^{\prime\prime}$ used to determine the radial intervals for separate evaluations of thickness. The images are plotted as log of the intensity. The scale bar indicates 1 kpc.
 (*fig:hacircles*)

</div>
<div id="div_fig3">

<img src="tmp_2504.05430/./feast_m51_ha_circle_dr3_3_and_4.jpg" alt="Fig3" width="100%"/>

**Figure 3. -** Left: H$\alpha$ Intensity scans (with the indicated interval in units of $10^{-13}$ erg s$^{-1}$ cm$^{-2}$ arcsec$^{-2}$, top) and power spectra (in the square of these units, bottom) corresponding approximately to the positions of the circles in Fig. \ref{fig:hacircles}. Both are shifted vertically by arbitrary amounts for clarity.  The intensity ordinate is a linear scale with one unit shown; factors of 2 and 20 compressions are used for the 2nd and 3rd scans up from the bottom.  The PS ordinate shows the 10 unit scale, which means the PS covers a range of $10^{10}$ in that interval. The lengths of the scans increase with radius. Right: Intensity scans (top) and PS (bottom) also corresponding to radii near the circles in Fig. \ref{fig:hacircles}, but chosen to avoid strong sources. The PS in the lower left are irregular. For example, they are flat at low $\log k/k_0$ when there are exceptionally strong sources in the scans. When there are no strong sources, the PS (lower right) are more uniform and may be averaged to give a better composite PS.
 (*fig:intensity*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.05430"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

432  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
